In [1]:
import folium
import pandas as pd


In [3]:
# Import folium MarkerCluster plugin
from folium.plugins import MarkerCluster
# Import folium MousePosition plugin
from folium.plugins import MousePosition
# Import folium DivIcon plugin
from folium.features import DivIcon

In [8]:
URL = 'https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/datasets/spacex_launch_geo.csv'
spacex_df=pd.read_csv(URL)

In [10]:
spacex_df.head()

,Flight Number,Date,Time (UTC),Booster Version,Launch Site,Payload,Payload Mass (kg),Orbit,Customer,Landing Outcome,class,Lat,Long
0,1,2010-06-04,18:45:00,F9 v1.0 B0003,CCAFS LC-40,Dragon Spacecraft Qualification Unit,0.0,LEO,SpaceX,Failure (parachute),0,28.562302,-80.577356
1,2,2010-12-08,15:43:00,F9 v1.0 B0004,CCAFS LC-40,"Dragon demo flight C1, two CubeSats, barrel o...",0.0,LEO (ISS),NASA (COTS) NRO,Failure (parachute),0,28.562302,-80.577356
2,3,2012-05-22,7:44:00,F9 v1.0 B0005,CCAFS LC-40,Dragon demo flight C2+,525.0,LEO (ISS),NASA (COTS),No attempt,0,28.562302,-80.577356
3,4,2012-10-08,0:35:00,F9 v1.0 B0006,CCAFS LC-40,SpaceX CRS-1,500.0,LEO (ISS),NASA (CRS),No attempt,0,28.562302,-80.577356
4,5,2013-03-01,15:10:00,F9 v1.0 B0007,CCAFS LC-40,SpaceX CRS-2,677.0,LEO (ISS),NASA (CRS),No attempt,0,28.562302,-80.577356


In [12]:
spacex_df = spacex_df[['Launch Site', 'Lat', 'Long', 'class']]
launch_sites_df = spacex_df.groupby(['Launch Site'], as_index=False).first()
launch_sites_df = launch_sites_df[['Launch Site', 'Lat', 'Long']]
launch_sites_df

,Launch Site,Lat,Long
0,CCAFS LC-40,28.562302,-80.577356
1,CCAFS SLC-40,28.563197,-80.576820
2,KSC LC-39A,28.573255,-80.646895
3,VAFB SLC-4E,34.632834,-120.610745


In [14]:
nasa_coordinate = [29.559684888503615, -95.0830971930759]
site_map = folium.Map(location=nasa_coordinate, zoom_start=10)

In [16]:
# Create a blue circle at NASA Johnson Space Center's coordinate with a popup label showing its name
circle = folium.Circle(nasa_coordinate, radius=1000, color='#d35400', fill=True).add_child(folium.Popup('NASA Johnson Space Center'))
# Create a blue circle at NASA Johnson Space Center's coordinate with a icon showing its name
marker = folium.map.Marker(
    nasa_coordinate,
    # Create an icon as a text label
    icon=DivIcon(
        icon_size=(20,20),
        icon_anchor=(0,0),
        html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % 'NASA JSC',
        )
    )
site_map.add_child(circle)
site_map.add_child(marker)

In [110]:
# Initial the map
site_map = folium.Map(location=nasa_coordinate, zoom_start=4)
# For each launch site, add a Circle object based on its coordinate (Lat, Long) values. In addition, add Launch site name as a popup label
for index, launch_site in launch_sites_df.iterrows():
    coordinate=[launch_site['Lat'],launch_site['Long']]
    name=launch_site['Launch Site']
    folium.Circle(coordinate, radius=100, color='#000000', fill=True).add_child(folium.Popup(name)).add_to(site_map)
    folium.map.Marker(coordinate, icon=DivIcon(icon_size=(20,20),icon_anchor=(0,0), html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % name, )).add_to(site_map)

site_map

In [112]:
spacex_df.tail(10)

,Launch Site,Lat,Long,class,marker_color
46,KSC LC-39A,28.573255,-80.646895,1,green
47,KSC LC-39A,28.573255,-80.646895,1,green
48,KSC LC-39A,28.573255,-80.646895,1,green
49,CCAFS SLC-40,28.563197,-80.576820,1,green
50,CCAFS SLC-40,28.563197,-80.576820,1,green
51,CCAFS SLC-40,28.563197,-80.576820,0,red
52,CCAFS SLC-40,28.563197,-80.576820,0,red
53,CCAFS SLC-40,28.563197,-80.576820,0,red
54,CCAFS SLC-40,28.563197,-80.576820,1,green
55,CCAFS SLC-40,28.563197,-80.576820,0,red


In [116]:
marker_cluster = MarkerCluster()

In [118]:
color_map = {1: 'green', 0: 'red'}
spacex_df['marker_color'] = spacex_df['class'].replace(color_map)

In [120]:
spacex_df.head()

,Launch Site,Lat,Long,class,marker_color
0,CCAFS LC-40,28.562302,-80.577356,0,red
1,CCAFS LC-40,28.562302,-80.577356,0,red
2,CCAFS LC-40,28.562302,-80.577356,0,red
3,CCAFS LC-40,28.562302,-80.577356,0,red
4,CCAFS LC-40,28.562302,-80.577356,0,red


In [122]:

for index, record in spacex_df.iterrows():

    coordinate = [record['Lat'], record['Long']]
    
    marker = folium.Marker(
        location=coordinate,
        icon=folium.Icon(color='white', icon_color=record['marker_color']),
        popup=f"Launch Site: {record['Launch Site']}<br>Class: {record['class']}"
    )
        
    marker_cluster.add_child(marker)

site_map.add_child(marker_cluster)

site_map

In [124]:
formatter = "function(num) {return L.Util.formatNum(num, 5);};"
mouse_position = MousePosition(
    position='topright',
    separator=' Long: ',
    empty_string='NaN',
    lng_first=False,
    num_digits=20,
    prefix='Lat:',
    lat_formatter=formatter,
    lng_formatter=formatter,
)

site_map.add_child(mouse_position)
site_map

In [126]:
from math import sin, cos, sqrt, atan2, radians

def calculate_distance(lat1, lon1, lat2, lon2):
    # approximate radius of earth in km
    R = 6373.0

    lat1 = radians(lat1)
    lon1 = radians(lon1)
    lat2 = radians(lat2)
    lon2 = radians(lon2)

    dlon = lon2 - lon1
    dlat = lat2 - lat1

    a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))

    distance = R * c
    return distance

In [128]:
launch_site_lat, launch_site_lon = 28.563, -80.57693

coastline_lat, coastline_lon = 28.56336, -80.56783

distance_coastline = calculate_distance(launch_site_lat, launch_site_lon, coastline_lat, coastline_lon)

launch_site_marker = folium.Marker(
    [launch_site_lat, launch_site_lon],
    popup="Launch Site",
    icon=folium.Icon(color='blue', icon='info-sign')
)
site_map.add_child(launch_site_marker)

coastline_marker = folium.Marker(
    [coastline_lat, coastline_lon],
    popup="Coastline Point",
    icon=folium.Icon(color='green', icon='info-sign')
)
site_map.add_child(coastline_marker)

distance_marker = folium.Marker(
    [(launch_site_lat + coastline_lat) / 2, (launch_site_lon + coastline_lon) / 2],  # Midpoint for the distance marker
    icon=DivIcon(
        icon_size=(20, 20),
        icon_anchor=(0, 0),
        html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % "{:10.2f} KM".format(distance_coastline),
    )
)
site_map.add_child(distance_marker)


site_map


In [130]:
coordinates1=[[launch_site_lat, launch_site_lon],[coastline_lat, coastline_lon]]
lines=folium.PolyLine(locations=coordinates1, weight=1)
site_map.add_child(lines)
site_map

In [132]:
highway_lat, highway_lon=28.56319,-80.57073
railway_lat, railway_lon=28.57115,-80.58543
city_lat, city_lon=28.07713,-80.606

distance_highway = calculate_distance(launch_site_lat, launch_site_lon, highway_lat, highway_lon)
distance_railway = calculate_distance(launch_site_lat, launch_site_lon, railway_lat, railway_lon)
distance_city = calculate_distance(launch_site_lat, launch_site_lon, city_lat, city_lon)

highway_marker = folium.Marker(
    [highway_lat, highway_lon],
    popup="Highway Point",
    icon=folium.Icon(color='black', icon='info-sign')
)
site_map.add_child(highway_marker)

distance_marker1 = folium.Marker(
    [(launch_site_lat + highway_lat) / 2, (launch_site_lon + highway_lon) / 2],  # Midpoint for the distance marker
    icon=DivIcon(
        icon_size=(20, 20),
        icon_anchor=(0, 0),
        html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % "{:10.2f} KM".format(distance_highway),
    )
)
site_map.add_child(distance_marker1)

coordinates2=[[launch_site_lat, launch_site_lon],[highway_lat, highway_lon]]
lines1=folium.PolyLine(locations=coordinates2, weight=1)
site_map.add_child(lines1)


railway_marker = folium.Marker(
    [railway_lat, railway_lon],
    popup="Railway Point",
    icon=folium.Icon(color='orange', icon='info-sign')
)
site_map.add_child(railway_marker)

distance_marker2 = folium.Marker(
    [(launch_site_lat + railway_lat) / 2, (launch_site_lon + railway_lon) / 2],  # Midpoint for the distance marker
    icon=DivIcon(
        icon_size=(20, 20),
        icon_anchor=(0, 0),
        html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % "{:10.2f} KM".format(distance_railway),
    )
)
site_map.add_child(distance_marker2)

coordinates3=[[launch_site_lat, launch_site_lon],[railway_lat, railway_lon]]
lines2=folium.PolyLine(locations=coordinates3, weight=1)
site_map.add_child(lines2)


city_marker = folium.Marker(
    [city_lat, city_lon],
    popup="City Point",
    icon=folium.Icon(color='purple', icon='info-sign')
)
site_map.add_child(city_marker)

distance_marker3 = folium.Marker(
    [(launch_site_lat + city_lat) / 2, (launch_site_lon + city_lon) / 2],  # Midpoint for the distance marker
    icon=DivIcon(
        icon_size=(20, 20),
        icon_anchor=(0, 0),
        html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % "{:10.2f} KM".format(distance_city),
    )
)
site_map.add_child(distance_marker3)

coordinates4=[[launch_site_lat, launch_site_lon],[city_lat, city_lon]]
lines3=folium.PolyLine(locations=coordinates4, weight=1)
site_map.add_child(lines3)

site_map